In [30]:
#install.packages('jsonlite')
library(jsonlite)
S5 <- fromJSON("http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk")
head(S5)

#S5-mgm4804308.3
#http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk
#s13_R2- https://api-ui.mg-rast.org/metagenome/72d6f57b296d676d343638363631342e33?verbosity=stats&detail=ontology&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk

#http://api.mg-rast.org/api.html#metagenome
#http://api.metagenomics.anl.gov/1/annotation/sequence/mmgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk

ERROR: Error in fromJSON("http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk"): unexpected character 'h'



In [31]:
library(rjson)
S5 <- rjson::fromJSON(file="http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk")
head(S5)

Warning message in file(con, "r"):
“cannot open URL 'http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk': HTTP status was '401 Unauthorized'”

ERROR: Error in file(con, "r"): cannot open the connection to 'http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk'


In [32]:
dAyu<-read.table('http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk',sep='')

Warning message in file(file, "rt"):
“cannot open URL 'http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk': HTTP status was '401 Unauthorized'”

ERROR: Error in file(file, "rt"): cannot open the connection to 'http://api.metagenomics.anl.gov/1/annotation/sequence/mgm4804308.3?evalue=10&type=organism&source=SwissProt&auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk'


In [ ]:
http://api.metagenomics.anl.gov/metadata/export/mgp17042?auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk
http://api.metagenomics.anl.gov/project/mgp17042?auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk
http://api.metagenomics.anl.gov/inbox?auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk
http://api.metagenomics.anl.gov/sample/mgm4804308.3?auth=kbm2R6jxx9LSnLVNmtawtFBVtVNbk

https://www.mg-rast.org/mgmain.html?mgpage=project&project=760ca003346d67703137303432 
mgp17042

https://www.mg-rast.org/mgmain.html?mgpage=pipeline

In [ ]:
Name    class   fwd     rev
s13     Vatta   TGGAACAA        TGGAACAA
S20-2   Pita    TGGCTTCA        TGGCTTCA
S23     Pita    TGGTGGTA        TGGTGGTA
S27     Kapha   TTCACGCA        TTCACGCA
S28     Vatta   AACTCACC        AACTCACC
S2      Kapha   TCCGTCTA        TCCGTCTA
S30     Kapha   AAGAGATC        AAGAGATC
S31     Kapha   AAGGACAC        AAGGACAC
S33     Kapha   AATCCGTC        AATCCGTC
S35     Vatta   AATGTTGC        AATGTTGC
S36     Vatta   ACACGACC        ACACGACC
S37     Pita    ACAGATTC        ACAGATTC
S3      Vatta   TCTTCACA        TCTTCACA
S40     Kapha   AGATGTAC        AGATGTAC
S44     Vatta   AGCACCTC        AGCACCTC
S46     Kapha   AGCCATGC        AGCCATGC
S47     Vatta   AGGCTAAC        AGGCTAAC
S48     Vatta   ATAGCGAC        ATAGCGAC
S5      Pita    TGAAGAGA        TGAAGAGA


In [12]:
system("mkdir ~/R/")
system("mkdir ~/R/libs/")


In [14]:
system("echo 'R_LIBS_USER=\"~/R/library\"' >  $HOME/.Renviron")

In [20]:
install.packages('devtools',lib.loc="/home/notebook/R/library")
install.packages('devtools')

Installing package into ‘/usr/local/spark-2.3.0-bin-hadoop2.7/R/lib’
(as ‘lib’ is unspecified)
Warning message in install.packages("devtools", lib.loc = "/home/notebook/R/library"):
“'lib = "/usr/local/spark-2.3.0-bin-hadoop2.7/R/lib"' is not writable”

ERROR: Error in install.packages("devtools", lib.loc = "/home/notebook/R/library"): unable to install packages
